In [2]:
import keras
import numpy as np

# Load the text data
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

# Parameters
maxlen = 60
step = 3

# Extract sequences and next characters
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# Get unique characters and their indices
chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
print('Unique characters:', len(chars))

# Vectorize sequences and next characters
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print('Shape of x:', x.shape)
print('Shape of y:', y.shape)

Corpus length: 600893
Number of sequences: 200278
Unique characters: 57
Shape of x: (200278, 60, 57)
Shape of y: (200278, 57)


In [7]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

model = Sequential([
    LSTM(128, input_shape = (maxlen, len(chars))),
    Dense(len(chars), activation = 'softmax')
])
optimizer = keras.optimizers.RMSprop(learning_rate = 0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

2024-07-25 14:52:04.545941: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-07-25 14:52:04.546131: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-07-25 14:52:04.546154: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-07-25 14:52:04.546861: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-25 14:52:04.547394: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/yasserali/anaconda3/envs/tf_m1/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Se

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random
import sys

# Text generation function
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Text generation function
def generate_text(model, seed_text, length, temperature=1.0):
    generated = ''
    sentence = seed_text.lower()
    generated += sentence
    print(f'Generating with seed: "{sentence}"')
    
    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.0
        
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()
    return generated

# Train the model and generate text at each epoch
for epoch in range(1, 60):
    print(f'\nEpoch {epoch}')
    model.fit(x, y, batch_size=128, epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    seed_text = text[start_index: start_index + maxlen]
    print(f'--- Generating with seed: "{seed_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'\n--- Temperature: {temperature} ---')
        generated_text = generate_text(model, seed_text, length=400, temperature=temperature)
        sys.stdout.write(generated_text)
        sys.stdout.flush()


Epoch 1
1565/1565 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - loss: 1.5412
--- Generating with seed: "o
much when she is obliged to answer. and after all, truth i"

--- Temperature: 0.2 ---
Generating with seed: "o
much when she is obliged to answer. and after all, truth i"
n a man and the fact and the sension of the sunscience of the sort of the find in the fact and something for the fact and the fact of the concernation of the individuals of the man and far the for the fact and such a such a sension of the sension of the fact of the survignt of the fact and the for the sension of the conception of the great the sension of the fact and the sension of the fact of eve
o
much when she is obliged to answer. and after all, truth in a man and the fact and the sension of the sunscience of the sort of the find in the fact and something for the fact and the fact of the concernation of the individuals of the man and far the for the fact and such a such a sension of the sension of the fact of the surv